In [1]:
!pip install fairlearn

     |████████████████████████████████| 24.6 MB 47 kB/s  eta 0:00:01


In [10]:
import pandas as pd
import numpy as np

## Load data and preprocess

In [14]:
data = pd.read_csv('fairness/data/preprocessed/adult_numerical.csv', sep=',')

In [15]:
# remove individuals in 'Other' race category
data = data[data['race'] != 'Other']

In [16]:
data['A_race'] = data['race'].astype("category").cat.codes
data['A_sex'] = data['sex'].astype("category").cat.codes

In [17]:
# one hot encode race and sex
data = pd.get_dummies(data, columns = ['race', 'sex'])

In [18]:
# remove redundant columns
data = data.drop(columns=['education-num', 'sex_Female', 'race_Amer-Indian-Eskimo',
                   'workclass_Without-pay', 'education_1st-4th', 'marital-status_Never-married',
                  'occupation_Other-service', 'relationship_Other-relative', 'native-country_Yugoslavia'])

In [19]:
# make income-per-year binary
data['Y'] = (data['income-per-year'] != '<=50K')

In [20]:
data = data.drop(columns=['income-per-year'])

In [21]:
# get convert each combination of race and sex to a numerical category
data['A_race-sex'] = data['race-sex'].astype("category").cat.codes

In [22]:
data = data.drop(columns=['race-sex'])

In [23]:
data.head()

,age,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,...,native-country_United-States,native-country_Vietnam,A_race,A_sex,race_Asian-Pac-Islander,race_Black,race_White,sex_Male,Y,A_race-sex
0,39,2174,0,40,0,0,0,0,0,1,...,1,0,3,1,0,0,1,1,False,7
1,50,0,0,13,0,0,0,0,1,0,...,1,0,3,1,0,0,1,1,False,7
2,38,0,0,40,0,0,1,0,0,0,...,1,0,3,1,0,0,1,1,False,7
3,53,0,0,40,0,0,1,0,0,0,...,1,0,2,1,0,1,0,1,False,5
4,28,0,0,40,0,0,1,0,0,0,...,0,0,2,0,0,1,0,0,False,4


In [13]:
# shuffle rows for randomization
data = data.sample(frac=1)

In [14]:
# separate data into 50% train and 50% test set
sep = int(0.50 * len(data) + 0.5)
train_data = data[:sep]
test_data = data[sep:]

In [15]:
print(len(train_data))
print(len(test_data))

14966
14965


In [16]:
X_train = train_data.drop(columns=['Y', 'A_race', 'A_sex', 'A_race-sex'])
X_test = test_data.drop(columns=['Y', 'A_race', 'A_sex', 'A_race-sex'])
Y_train = train_data['Y']
Y_test = test_data['Y']
A_race_train = train_data['A_race']
A_race_test = test_data['A_race']
A_sex_train = train_data['A_sex']
A_sex_test = test_data['A_sex']
A_race_sex_train = train_data['A_race-sex']
A_race_sex_test = test_data['A_race-sex']

# Feldman et al. Repair (Disparate Impact)

In [4]:
!pip install aif360

     |████████████████████████████████| 175 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 356 kB 9.2 MB/s eta 0:00:01
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30182 sha256=a16209c9426150376dd0acb398f252d68d373b78f9ed26c130f15c551415f201
  Stored in directory: /Users/carinalewandowski/Library/Caches/pip/wheels/6a/37/3e/d9e8ebaf73956a3ebd2ee41869444dbd2a702d7142bcf93c42
  Created wheel for shap: filename=shap-0.39.0-cp38-cp38-macosx_10_9_x86_64.whl size=412981 sha256=d3d79015b4bb03880945e1c3434e84b3dfa8ec1d55beed4c2d832ad8c908640f
  Stored in directory: /Users/carinalewandowski/Library/Caches/pip/wheels/3d/c9/06/734ed80d6d61fad331974bf62017b4ea6b33488082b9f5e67e
Successfully built memory-profiler shap


In [125]:
import aif360.algorithms.preprocessing as AIF
from aif360.datasets import AdultDataset

In [126]:
ad = AdultDataset(instance_weights_name='fnlwgt', features_to_drop=[])

In [47]:
# instantiate dataset 
'''
single_protected = ['sex']
single_privileged = [['Male']]
ad = AdultDataset(protected_attribute_names=single_protected, privileged_classes=single_privileged,
                  categorical_features=[],
                  features_to_keep=['age', 'education-num'])
# check
print(ad.feature_names)
print(ad.label_names)
'''

"\nsingle_protected = ['sex']\nsingle_privileged = [['Male']]\nad = AdultDataset(protected_attribute_names=single_protected, privileged_classes=single_privileged,\n                  categorical_features=[],\n                  features_to_keep=['age', 'education-num'])\n# check\nprint(ad.feature_names)\nprint(ad.label_names)\n"

In [48]:
# keep track of mapping from float -> str for proetected attributes and/or labels
# use to modify mapping in 'metadata'
'''
label_map = {1.0: '>50K', 0.0: '<=50K'}
protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
ad = AdultDataset(protected_attribute_names=['sex'],
                  privileged_classes=[['Male']], metadata={'label_map': label_map,
                                                           'protected_attribute_maps': protected_attribute_maps})
'''

"\nlabel_map = {1.0: '>50K', 0.0: '<=50K'}\nprotected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]\nad = AdultDataset(protected_attribute_names=['sex'],\n                  privileged_classes=[['Male']], metadata={'label_map': label_map,\n                                                           'protected_attribute_maps': protected_attribute_maps})\n"

In [127]:
repairer = AIF.DisparateImpactRemover(repair_level=1.0, sensitive_attribute='race')
repaired_data = repairer.fit_transform(ad)

In [128]:
repaired_df, repaired_attrs = repaired_data.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)
ad_df, ad_attrs = ad.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=True)

In [131]:
ad_df

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year
0,25.0,7.0,0.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,9.0,1.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,12.0,1.0,1.0,0.0,0.0,40.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,44.0,10.0,0.0,1.0,7688.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,34.0,6.0,1.0,1.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27.0,12.0,1.0,0.0,0.0,0.0,38.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,40.0,9.0,1.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
48839,58.0,9.0,1.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48840,22.0,9.0,1.0,1.0,0.0,0.0,20.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [132]:
from sklearn.linear_model import LogisticRegression
from copy import deepcopy

In [134]:
Y_rep = np.array(repaired_df[['income-per-year']]).reshape(((len(repaired_df),)))
Xs_rep = np.array(repaired_df.drop(columns='income-per-year'))
Y = np.array(ad_df[['income-per-year']]).reshape(((len(ad_df),)))
Xs = np.array(ad_df.drop(columns='income-per-year'))

In [138]:
Xs.shape

(45222, 98)

In [139]:
clf = LogisticRegression(max_iter = 300).fit(Xs, Y)
clf_rep = LogisticRegression(max_iter = 300).fit(Xs_rep, Y_rep)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [141]:
acc = clf.score(Xs, Y)
predicted_labels = clf.predict(Xs).reshape((len(Y), 1))
print('original accuracy: ', acc)

acc_rep = clf_rep.score(Xs_rep, Y_rep)
predicted_labels_rep = clf_rep.predict(Xs_rep).reshape((len(Y_rep), 1))
print('accuracy on repaired data: ', acc_rep)

original accuracy:  0.8440582017602052
accuracy on repaired data:  0.8431957896599


In [142]:
ad_pred = deepcopy(ad)
ad_pred.labels = predicted_labels

ad_pred_rep = deepcopy(ad)
ad_pred_rep.labels = predicted_labels_rep

In [143]:
from aif360.metrics import ClassificationMetric

In [147]:
u = [{'race': 0}]
p = [{'race': 1}]
metrics = ClassificationMetric(ad,ad_pred,unprivileged_groups=u, privileged_groups=p)
DI = metrics.disparate_impact()
print('Original Disparate Impact:', DI)

Original Disparate Impact: 0.5753559616317514


In [148]:
metrics_rep = ClassificationMetric(ad,ad_pred_rep,unprivileged_groups=u, privileged_groups=p)
DI_rep = metrics_rep.disparate_impact()
print('Disparate Impact on Repaired Data:', DI_rep)

Disparate Impact on Repaired Data: 0.690573938777518


# Kamishima et al. Regularization (Prejudice Remover)

In [150]:
!pip install tensorflow

     |████████████████████████████████| 173.9 MB 246 bytes/s a 0:00:01  |▏                               | 849 kB 3.2 MB/s eta 0:00:54     |▎                               | 1.4 MB 3.2 MB/s eta 0:00:54     |██▏                             | 12.0 MB 42.2 MB/s eta 0:00:04     |████▌                           | 24.3 MB 42.2 MB/s eta 0:00:04     |██████▏                         | 33.3 MB 72.6 MB/s eta 0:00:02     |████████                        | 43.4 MB 2.2 MB/s eta 0:01:01     |███████████▍                    | 61.7 MB 8.0 MB/s eta 0:00:14     |███████████▊                    | 63.5 MB 8.0 MB/s eta 0:00:14     |███████████▉                    | 64.4 MB 8.0 MB/s eta 0:00:14     |██████████████▏                 | 77.1 MB 4.7 MB/s eta 0:00:21     |██████████████▊                 | 80.1 MB 4.7 MB/s eta 0:00:20 |███████████████                 | 81.9 MB 4.7 MB/s eta 0:00:20     |████████████████▉               | 91.3 MB 4.6 MB/s eta 0:00:19     |█████████████████▌              | 95.1 MB 4.6 

In [151]:
import aif360.algorithms.inprocessing as AIF_inp

In [152]:
PrejRemover = AIF_inp.PrejudiceRemover(eta=1.0, sensitive_attr='race', class_attr='income-per-year')

In [153]:
kamishima_data = PrejRemover.fit_predict(ad)

In [155]:
metrics_kamishima = ClassificationMetric(ad,kamishima_data,unprivileged_groups=u, privileged_groups=p)
DI_kamishima = metrics_kamishima.disparate_impact()
print('Disparate Impact on Repaired Data:', DI_kamishima)

Disparate Impact on Repaired Data: 0.5496994960493873


## Fit Logistic Regression Model with Fairness Constraints

In [61]:
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import GridSearch, EqualizedOdds
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)

from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [62]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(roc_auc_score, y_true, x, sensitive_features=group).difference(method='between_groups'), False),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

In [74]:
# Fit logistic regression model
model = LogisticRegression(max_iter=500, multi_class='ovr')
model.fit(X_train, Y_train)

/home/sophie/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=500, multi_class='ovr')

In [79]:
print('Unconstrained model accuracies:')
train_acc, test_acc = model.score(X_train, Y_train), model.score(X_test, Y_test)
unconstrained_test_preds = model.predict(X_test)
unconstrained_test_scores = model.predict_proba(X_test)[:, 1]
print('Train acc:', train_acc)
print('Test acc:', test_acc)
print('Overall generalization gap:', train_acc - test_acc)

Unconstrained model accuracies:
Train acc: 0.8427101429907791
Test acc: 0.8471099231540261
Overall generalization gap: -0.004399780163247047


### Equalized Odds

#### Using postprocessing algorithm from Hardt et. al "Equality of Opportunity in Supervised Learning"


In [81]:
postprocess_est = ThresholdOptimizer(
    estimator=model,
    constraints="equalized_odds",
    prefit=True)

In [82]:
postprocess_est.fit(X_train, Y_train, sensitive_features=A_sex_train)

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=LogisticRegression(max_iter=500,
                                                multi_class='ovr'),
                   prefit=True)

In [83]:
postprocess_preds_train = postprocess_est.predict(X_train, sensitive_features=A_sex_train)
postprocess_preds_test = postprocess_est.predict(X_test, sensitive_features=A_sex_test)

In [86]:
print('Hardt et al model accuracies:')

train_acc = sum(postprocess_preds_train != Y_train) / len(postprocess_preds_train)
test_acc = sum(postprocess_preds_test != Y_test) / len(postprocess_preds_test)
print('Train acc:', train_acc)
print('Test acc:', test_acc)
print('Overall generalization gap:', train_acc - test_acc)

Hardt et al model accuracies:
Train acc: 0.19256982493652278
Test acc: 0.18757099899766122
Overall generalization gap: 0.004998825938861556


In [87]:
models_dict = { 'Unconstrained': (unconstrained_test_preds, unconstrained_test_scores),
    'Hardt et al.': (postprocess_preds_test, postprocess_preds_test)}
get_metrics_df(models_dict, Y_test, A_race_sex_test)

,Unconstrained,Hardt et al.
Overall selection rate,0.207551,0.196525
Demographic parity difference,0.2987,0.203715
Demographic parity ratio,0.0787458,0.28086
------,,
False positive rate difference,0.159132,0.119589
False negative rate difference,0.477778,0.548611
Equalized odds difference,0.477778,0.548611
------,,
Overall AUC,0.899805,0.713856
AUC difference,0.136121,0.258223
